In [1]:
!pip3 install flask

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [2]:
from flask import Flask, jsonify, request
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from yahoo_fin import options as op

In [3]:
app = Flask(__name__)

In [4]:
@app.route("/")
def simulator_service():
    return "Up & Running !"


@app.route("/mcs/<string:option_type>")
def derivative_option(option_type):
    result = {
        "simulator type":"Monte Carlo Simulation",
        "Option type":option_type
    }
    return jsonify(result)

@app.route("/mcs/price/<spotPrice>/<strikePrice>/<time>/<volatility>/<steps>/<trials>")
def calculate_mcs(spotPrice,strikePrice,time,volatility,steps,trials):
       
        spotPrice = float(spotPrice)
        strikePrice = float(strikePrice)
        T = int(time)
        volatility = float(volatility)
        steps = int(steps)
        trials = int(trials)
        
        r = 0
        q = 0;
        
        paths= geo_paths(spotPrice,T,r,q,volatility,steps,trials)
        payoffs = np.maximum(paths[-1]-strikePrice, 0)
        option_price = np.mean(payoffs)*np.exp(-r*T)
        
        return jsonify(option_price) # return data with 200 OK

In [5]:
def geo_paths(S, T, r, q, sigma, steps, N):
    """
    Inputs
    #S = Current stock Price
    #K = Strike Price
    #T = Time to maturity 1 year = 1, 1 months = 1/12
    #r = risk free interest rate
    #q = dividend yield
    # sigma = volatility 
    
    Output
    # [steps,N] Matrix of asset paths 
    """
    dt = T/steps
    ST = np.log(S) +  np.cumsum(((r - q - sigma**2/2)*dt +\
                              sigma*np.sqrt(dt) * \
                              np.random.normal(size=(steps,N))),axis=0)
    
    return np.exp(ST)

In [6]:
@app.route("/symbols")
def get_symbols():
    sp_wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    sp_wiki_df_list = pd.read_html(sp_wiki_url)
    sp_df = sp_wiki_df_list[0]
    return jsonify(list(sp_df['Symbol'].values))

In [7]:
ticker = 'AAPL'
expDate = op.get_expiration_dates(ticker)
callsData = op.get_calls(ticker,expDate[0]).head(1)
print("for ticker => ",callsData)

for ticker =>           Contract Name        Last Trade Date  Strike  Last Price   Bid  Ask  \
0  AAPL230721C00050000  2023-07-17 3:22PM EDT    50.0      144.15  0.00  0.0   

   Change % Change  Volume  Open Interest Implied Volatility  
0     0.0        -       2              0              0.00%  


In [8]:
callsData.columns

Index(['Contract Name', 'Last Trade Date', 'Strike', 'Last Price', 'Bid',
       'Ask', 'Change', '% Change', 'Volume', 'Open Interest',
       'Implied Volatility'],
      dtype='object')

In [9]:
@app.route("/symbol/details/<ticker>")
def get_ticker_details(ticker):
    expDate = op.get_expiration_dates(ticker)
    callsData = op.get_calls(ticker,expDate[0]).head(1)
    return callsData.to_json()

In [ ]:
if __name__ == '__main__':
    app.run()  # run our Flask app
    #app.run(host='0.0.0.0',port=8080)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Jul/2023 04:56:02] "GET /symbol/details/AAPL HTTP/1.1" 200 -
